# Third Project


In this third project, you will do some analysis and optimizations in uCIR. First, you will determine the basic blocks. After, you're going to build a control flow graph (CFG) and compute the SSA graph, reflected through phi operations in the uCIR. Next, you're going to compute the dominators for the CFG, i.e. for each basic block, identify the set of basic blocks that dominate that block. This means that every path from the entry block to that block must go through the blocks in the dominator set. After that, you can simplify control flow (1) by merging consecutive blocks where the parent has one child, the child one parent, and both have compatible instruction leaders; (2) find all immediate dead blocks and remove then and finally, perform some basic optimizations like constant propagation and dead code elimination.

We will try to give you a hand by giving you some of the pieces prewritten and littered your code with helpful statements. By the time you’re done, you’ll have a pretty thorough understanding of the analyses and optimizations realized by real compilers.

## Basic Block Creation

After constructing the sequence of instructions in its intermediate representation for the program, we will determine the basic blocks. A basic block is a sequence of instructions where the control flow enters only at the beginning of the block and exits at the end of the block, without the possibility of deviation to any other part of the program. The basic blocks make up the nodes of a control flow graph (CFG), a structure that will be important for performing code optimizations.

To create the basic blocks, you must first determine the set of leaders, which will match the first statement of each basic block. To do this, proceed according to the algorithm below:

1. The first instruction is a leader;
2. Any instruction that is subject to conditional or unconditional deviation is a leader;
3. Any instruction that comes immediately after a conditional or unconditional deviation instruction is a leader;
4. For each leader, your basic block consists of the leader and all instructions that follow him to the next leader, excluding this last.

When building the basic blocks, you may need to include additional statements so that the first statement of a basic block is always a label and the last statement is a jump statement (conditional or unconditional).

### Sample Code

The sample code below defines classes and functions for creating and navigating
basic blocks.  You need to write all of the code needed yourself.

In [1]:
# An example of how to create basic blocks

class Block(object):
    def __init__(self):
        self.instructions = []   # Instructions in the block
        self.next_block =None    # Link to the next block

    def append(self,instr):
        self.instructions.append(instr)

    def __iter__(self):
        return iter(self.instructions)

class BasicBlock(Block):
    '''
    Class for a simple basic block.  Control flow unconditionally
    flows to the next block.
    '''
    pass

class IfBlock(Block):
    '''
    Class for a basic-block representing an if-else.  There are
    two branches to handle each possibility.
    '''
    def __init__(self):
        super(IfBlock,self).__init__()
        self.if_branch = None
        self.else_branch = None
        self.test = None

class WhileBlock(Block):
    def __init__(self):
        super(WhileBlock, self).__init__()
        self.test = None
        self.body = None

class BlockVisitor(object):
    '''
    Class for visiting basic blocks.  Define a subclass and define
    methods such as visit_BasicBlock or visit_IfBlock to implement
    custom processing (similar to ASTs).
    '''
    def visit(self,block):
        while isinstance(block,Block):
            name = "visit_%s" % type(block).__name__
            if hasattr(self,name):
                getattr(self,name)(block)
            block = block.next_block